In [2]:
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard

import os
import cv2
import random
import numpy as np
import pickle
import time

NAME = "Flare-detector-cnn-32x2-{}".format(int(time.time()))
# tensorboard = TensorBoard(log_dir=f'logs/{NAME}')

In [ ]:
DATADIR = "./training/"
CATEGORIES = ['flare','good']
training_data = []

In [ ]:
# print(img_array.shape)
# new_array = cv2.resize(img_array, (500, 400))
# plt.imshow(new_array)

In [ ]:
def create_training_data():
    training_data = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (500, 400))
                training_data.append([new_array, class_num])
            except Exception as e:
                print(f"Exception: {e}")
                
    return training_data

In [ ]:
training_data = create_training_data()

# shuffling the data
random.shuffle(training_data)

In [ ]:
# pack it into the variables we're going to use
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, 400, 500, 3)
y = np.array(y)

In [ ]:
pickle.dump(X, open("X.pickle","wb"))
pickle.dump(y, open("y.pickle","wb"))

In [ ]:
X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

In [ ]:
# We should scale the data / normalise it
X = X/255.0

In [36]:
batch_size = 32

# Building a simple stack of 2 convolution layers with a ReLU activation
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# On top adding two fully-connected layers. We end the model with a single
# unit and a sigmoid function, which is good for binary classification
model.add(Flatten()) # this converts our 3D feature  maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

adam = optimizers.Adam(learning_rate=0.0005)

model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

# model.fit(X, y, batch_size= batch_size, epochs=50, validation_split=0.2, callbacks=[tensorboard])
model.fit(X, y, batch_size= batch_size, epochs=50, validation_split=0.2)

pickle.dump(model, open("model.pickle", "wb"))

Train on 64 samples, validate on 16 samples
Epoch 1/50
64/64 [==============================] - 5s 82ms/sample - loss: 1.2074 - accuracy: 0.5156 - val_loss: 2.3632 - val_accuracy: 0.5625
Epoch 2/50
64/64 [==============================] - 5s 73ms/sample - loss: 1.8956 - accuracy: 0.6406 - val_loss: 1.4473 - val_accuracy: 0.4375
Epoch 3/50
64/64 [==============================] - 5s 72ms/sample - loss: 1.0110 - accuracy: 0.6094 - val_loss: 0.5024 - val_accuracy: 0.8125
Epoch 4/50
64/64 [==============================] - 5s 73ms/sample - loss: 0.4095 - accuracy: 0.8281 - val_loss: 0.6515 - val_accuracy: 0.6875
Epoch 5/50
64/64 [==============================] - 5s 71ms/sample - loss: 0.3327 - accuracy: 0.8750 - val_loss: 0.5023 - val_accuracy: 0.8125
Epoch 6/50
64/64 [==============================] - 5s 73ms/sample - loss: 0

TypeError: can't pickle _thread.RLock objects

In [37]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [38]:
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")



Saved model to disk


In [3]:
cv2.imread('G0014859.JPG')



array([[[254, 255, 251],
        [254, 255, 251],
        [254, 255, 253],
        ...,
        [254, 255, 245],
        [254, 255, 245],
        [254, 255, 245]],

       [[254, 255, 251],
        [253, 254, 250],
        [252, 253, 249],
        ...,
        [254, 255, 245],
        [254, 255, 245],
        [254, 255, 245]],

       [[253, 254, 250],
        [251, 252, 248],
        [251, 252, 248],
        ...,
        [254, 255, 245],
        [254, 255, 245],
        [254, 255, 245]],

       ...,

       [[192, 181, 167],
        [192, 181, 167],
        [191, 182, 168],
        ...,
        [212, 207, 209],
        [213, 208, 210],
        [213, 208, 210]],

       [[192, 181, 167],
        [191, 180, 166],
        [193, 182, 168],
        ...,
        [213, 208, 210],
        [213, 208, 210],
        [212, 207, 209]],

       [[192, 181, 167],
        [191, 180, 166],
        [191, 181, 164],
        ...,
        [212, 207, 208],
        [210, 207, 209],
        [208, 205, 207]]